In [1]:
#load data
import numpy as np
import pandas as pd 

#Load Data
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
    
import time
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px

from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax, skew, kurtosis

import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

In [2]:
#load data
train = pd.read_csv('../input/train.csv', index_col='Id')
test = pd.read_csv('../input/test.csv', index_col='Id')

X = pd.concat([train.iloc[:, :-1], test])
y = train.iloc[:, -1]

In [3]:
X.drop(['GarageYrBlt','TotRmsAbvGrd','GarageCars','PoolQC', 'MiscFeature', 'Alley', 'Fence','MoSold', 'YrSold'], axis = 1, inplace=True)

In [4]:
X['MSSubClass'] = X['MSSubClass'].astype('str')
cat_list = []
num_list = []

for var in X.columns:
    if X[var].dtype == 'object':
        cat_list.append(var)
    else:
        num_list.append(var)

# higly skewed categorical variables (>95%) are dropped

cat_drop_list = []

for var in cat_list:
    ratio = X[var].value_counts().iloc[0]/X.shape[0] * 100
    if ratio > 95:
        cat_drop_list.append(var)
        
print(cat_drop_list)
X.drop(cat_drop_list,axis = 1,inplace=True)

# for numerical variables that are highly skewed, try to make them Gaussian-like by box-cox transformation

num_trans_list = []

for var in num_list:
    ratio = X[var].value_counts().iloc[0]/X.shape[0] * 100
    if ratio > 95:
        num_trans_list.append(var)
        
print(num_trans_list)
for var in num_trans_list:
    X[var] = boxcox1p(X[var], boxcox_normmax(X[var] + 1))

['Street', 'Utilities', 'LandSlope', 'Condition2', 'RoofMatl', 'Heating']
['LowQualFinSF', 'KitchenAbvGr', '3SsnPorch', 'PoolArea', 'MiscVal']


In [5]:
train = train.drop(train[train['LotFrontage'] > 200].index)
train = train.drop(train[train['LotArea']> 100000].index)
train = train.drop(train[train['MasVnrArea'] > 1200].index)
train = train.drop(train[train['BsmtUnfSF'] > 2000].index)
train = train.drop(train[train['2ndFlrSF'] > 2000].index)
train = train.drop(train[train['GarageArea'] > 1130].index)
train = train.drop(train[train['WoodDeckSF'] > 600].index)
train = train.drop(train[train['OpenPorchSF'] > 400].index)
train = train.drop(train[train['BsmtFinSF1'] > 3000].index)
train = train.drop(train[train['TotalBsmtSF'] > 4000].index)
train = train.drop(train[train['GrLivArea'] > 4000].index)

In [6]:
# for ordinal variables
# There are some columns which are ordinal by nature, which represents the quality or condition of certain housing features. 
# In this case, we will map the respective strings to a value. The better the quality, the higher the value

ordinal_map = {'Ex': 5,'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA':0}
fintype_map = {'GLQ': 6,'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1, 'NA': 0}
expose_map = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}

ord_col = ['ExterQual','ExterCond','BsmtQual', 'BsmtCond','HeatingQC','KitchenQual','GarageQual','GarageCond', 'FireplaceQu']
X[ord_col] = X[ord_col].fillna('NA')

for var in ord_col:
    X[var] = X[var].map(ordinal_map)
    
BsmtFin_col = ['BsmtFinType1','BsmtFinType2']
X[BsmtFin_col] = X[BsmtFin_col].fillna('NA')
for var in BsmtFin_col:
    X[var] = X[var].map(fintype_map)

X['BsmtExposure'] = X['BsmtExposure'].fillna('NA')
X['BsmtExposure'] = X['BsmtExposure'].map(expose_map)

In [7]:
# for numerical variables

neigh_lot = X.groupby('Neighborhood')['LotFrontage'].median().reset_index(name = 'LotFrontage_median') 
neigh_garage = X.groupby('Neighborhood')['GarageArea'].median().reset_index(name = 'GarageArea_median')

In [8]:
# lot frontage and garage area medians are correlated to neighborhood, so fill their nas using respective medians

X['LotFrontage'] = X.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
X['GarageArea'] = X.groupby('Neighborhood')['GarageArea'].transform(lambda x: x.fillna(x.median()))

In [9]:
# to fill na using median for the rest of numerical variables

for var in ["BsmtHalfBath", "BsmtFullBath", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "MasVnrArea"]:
    X[var] = X[var].fillna(X[var].median())

In [10]:
# for categorical variables

X['GarageFinish'] = X['GarageFinish'].fillna('NA')
X['GarageType'] = X['GarageType'].fillna('NA')

In [11]:
# to fill missing values (categorical variables) with mode

cat_col = ['MasVnrType', 'MSZoning', 'Functional', 'Exterior2nd', 'Exterior1st', 'Electrical', 'SaleType']

for var in cat_col:
    X[var] = X[var].fillna(X[var].mode().iloc[0])

In [12]:
# feature engineering (by domain knowledge)

X['TotalLot'] = X['LotFrontage'] + X['LotArea']
X['TotalBsmtFin'] = X['BsmtFinSF1'] + X['BsmtFinSF2']
X['TotalSF'] = X['TotalBsmtSF'] + X['2ndFlrSF'] + X['1stFlrSF']
X['TotalBath'] = X['FullBath'] + X['HalfBath'] * 0.5 + X['BsmtFullBath'] + X['BsmtHalfBath'] * 0.5
X['TotalPorch'] = X['OpenPorchSF'] + X['EnclosedPorch'] + X['ScreenPorch'] + X['WoodDeckSF']

drop_list = [
    'LotFrontage', 'LotArea',
    'BsmtFinSF1', 'BsmtFinSF2',
    'TotalBsmtSF', '2ndFlrSF','1stFlrSF',
    'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath',
    'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'WoodDeckSF'
]

X.drop(drop_list,axis=1, inplace=True)

In [13]:
# binary columns

bin_col = ['MasVnrArea','TotalPorch','PoolArea', 'GarageQual', 'BsmtQual']

for var in bin_col:
    col_name = var + '_bin'
    X[col_name] = X[var].apply(lambda x: 1 if x > 0 else 0)

In [14]:
X = pd.get_dummies(X)

y = y.apply(lambda x: np.log1p(x))

In [15]:
# training data preparation

x = X.loc[train.index]
y = y.loc[train.index]

In [16]:
# xgboost model
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, RepeatedKFold
from xgboost import XGBRegressor

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', XGBRegressor(learning_rate = 0.1,
                                   max_depth = 6,
                                   min_child_weight = 10,
                                   subsample = 0.7,
                                   colsample_bytree= 1,
                                   reg_lambda = 1,
                                   gamma = 0.005,
                                   reg_alpha = 0.0005,
                                   n_estimators = 202)))

pipe_xgb = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_xgb, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())

average score: -0.0797
STD: 0.0070


In [17]:
# lightgbm model

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, RepeatedKFold
from lightgbm import LGBMRegressor

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', LGBMRegressor(
    n_estimators = 1800,
    min_child_samples = 6,
    max_depth = 5,
    learning_rate = 0.01,
    lambda_l2 = 0.19,
    lambda_l1 = 0.04,
    feature_fraction = 0.25
)))

pipe_lgbm = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_lgbm, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0751
STD: 0.0065
training time: 41.54


In [18]:
# catboost model

from catboost import CatBoostRegressor

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', CatBoostRegressor())) # the virgin model turned out to be the best one

pipe_cat = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_cat, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0752
STD: 0.0066
training time: 234.79


In [19]:
from sklearn.linear_model import ElasticNetCV

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', ElasticNetCV(cv = 3)))

pipe_en = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_en, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0765
STD: 0.0065
training time: 7.90


In [20]:
from sklearn.linear_model import LassoCV

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', LassoCV(n_alphas = 207)))

pipe_lasso = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_lasso, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0766
STD: 0.0065
training time: 13.55


In [21]:
from sklearn.linear_model import RidgeCV

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', RidgeCV(cv=7)))

pipe_ridge = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_ridge, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0790
STD: 0.0070
training time: 5.52


In [22]:
from sklearn.svm import SVR

steps = list()
steps.append(('std', RobustScaler()))
steps.append(('norm', MinMaxScaler()))
steps.append(('model', SVR(epsilon=0.001,
                          gamma=0.028300001,
                          C=1)))

pipe_svr = Pipeline(steps=steps)

cv = RepeatedKFold(n_repeats=5, n_splits=10,random_state=1)
start = time.time()
scores = cross_val_score(pipe_svr, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0775
STD: 0.0062
training time: 15.33


In [23]:
### stacking techqiues
# stacking solo is not superior to blending
# time-consuming

from mlxtend.regressor import StackingCVRegressor

stack_all = StackingCVRegressor(regressors= (pipe_xgb, pipe_lgbm, pipe_cat,
                                            pipe_en, pipe_lasso, pipe_svr),
                               meta_regressor = pipe_lgbm,
                               use_features_in_secondary=True,
                               )
start = time.time()
scores = cross_val_score(stack_all, x, y, scoring= 'neg_mean_absolute_error', cv = 3,
                        n_jobs = -1)
stop = time.time()

print('average score: %.4f' % scores.mean())
print('STD: %.4f' % scores.std())
print('training time: %.2f' % (stop - start))

average score: -0.0772
STD: 0.0026
training time: 602.50


In [24]:
# blending techniques: 

def pipe_blend_fit_predict(X, b, c, d, e, f, g, h, i):
    
    pipe_xgb.fit(x, y)
    pipe_lgbm.fit(x, y)
    pipe_cat.fit(x, y)
    pipe_en.fit(x,y)
    pipe_lasso.fit(x,y)
    pipe_ridge.fit(x,y)
    pipe_svr.fit(x,y)
    stack_all.fit(x, y)
    
    return ((b * pipe_xgb.predict(X)) + 
            (c * pipe_lgbm.predict(X)) + 
            (d * pipe_cat.predict(X)) + 
            (e * pipe_en.predict(X)) + 
            (f * pipe_lasso.predict(X)) + 
            (g * pipe_ridge.predict(X).flatten()) + 
            (h * pipe_svr.predict(X)) + 
            (i * stack_all.predict(X)))

In [25]:
test_df = X.loc[test.index]

pred = np.exp(
    pipe_blend_fit_predict(test_df, 0.1, 0.20, 0.15, 0.1, 0.1, 0.1, 0.1, 0.15)
) 

# blending led to the best result, compared to individual models (not included though)
# model weights were chosen arbitrarily: empirically, larger weights assigned to more accurate models, such as catboost and stacked models -- higher scores obtianed

submission = pd.DataFrame({
    'Id': test.index,
    'SalePrice': pred
})

submission.to_csv('submission.csv', index=False)

[LightGBM] [Warning] lambda_l1 is set=0.04, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.04
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] lambda_l2 is set=0.19, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.19
Learning rate set to 0.041591
0:	learn: 0.3756888	total: 50.9ms	remaining: 50.8s
1:	learn: 0.3652073	total: 54.4ms	remaining: 27.2s
2:	learn: 0.3550741	total: 58.3ms	remaining: 19.4s
3:	learn: 0.3458313	total: 62ms	remaining: 15.4s
4:	learn: 0.3370824	total: 66.3ms	remaining: 13.2s
5:	learn: 0.3288666	total: 70.6ms	remaining: 11.7s
6:	learn: 0.3208881	total: 74.1ms	remaining: 10.5s
7:	learn: 0.3133138	total: 77.9ms	remaining: 9.66s
8:	learn: 0.3060459	total: 82.2ms	remaining: 9.05s
9:	learn: 0.2987655	total: 86.1ms	remaining: 8.53s
10:	learn: 0.2911144	total: 89.9ms	remaining: 8.08s
11:	learn: 0.2843118	total: 93.3ms	remaining: 7.68s
12:	learn: 0.2772748	